# Goal:

For the sales, I'd like to denote whether it was sold at a loss or a gain with a red or green icon. The rest of the info could be put into a pop up box, if that works

oh and btw maybe put listings in blue or something so we can see that they are not sold yet

# Imports

In [8]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import textwrap

## Data read-in

In [2]:
# df = pd.read_clipboard()
# df.to_csv("new_sales_listings.csv")

In [3]:
df = pd.read_csv("Miami Beach Office Projects - Sheet1.csv")

In [4]:
df

,Name of Project,Developer(s),coordinates,Status,Caption,Total square footage,Office square footage,Office space preleased,Link
0,Giller Tower,Giller family,"25.812843946932976, -80.13520530046938","In June, the Miami Beach Design Review Board a...",The Giller family plans a mixed-use project on...,NaN,NaN,NaN,https://therealdeal.com/miami/2024/06/05/ira-g...
1,The Fifth Miami Beach,"Sumaida + Khurana, Bizzi+Bilgili. Former Googl...","25.774032963985963, -80.13801069619636",Under construction. Completion is expected nex...,Sumaida + Khurana and Bizzi+Bilgili are develo...,"60,200 sf",NaN,"About 10,000 sf",https://therealdeal.com/miami/2023/12/27/bizzi...
2,NaN,"Sumaida + Khurana, with partners","25.773892628650724, -80.13974700627581","On April 25, the Miami Beach Planning Board vo...",Sumaida + Khurana also plans a five-story buil...,NaN,NaN,NaN,https://therealdeal.com/miami/2024/03/28/sumai...
3,One Soundscape Park,Shvo,"25.791455592403146, -80.13144604965949",Shvo's planned five-story One Soundscape Park ...,Shvo's planned five-story One Soundscape Park ...,"62,500 sf",NaN,NaN,https://therealdeal.com/miami/2024/02/01/micha...
4,The Alton,Shvo,"25.791265819966448, -80.14159472805648",It scored final approval in December.,Shvo's six-story The Alton mixed-use building ...,NaN,"170,000 sf",NaN,https://therealdeal.com/miami/2023/10/11/micha...
5,NaN,"Wayne M. Boich, Bruce Beal and Andrew Mathias","25.79516704153476, -80.14165522529154",NaN,"Wayne M. Boich, in partnership with Bruce Beal...",NaN,NaN,NaN,https://therealdeal.com/miami/2022/12/14/miami...
6,Eighteen Sunset,"Deco Capital Group, led by Bradley Colmer, and...","25.792690596928665, -80.14414600113716",Under construction,Deco Capital Group and RWN Real Estate Partner...,NaN,NaN,NaN,https://therealdeal.com/miami/2021/05/14/deco-...


## Geocode

In [7]:
# Convert the "coordinates" column to tuples of floats
df['coordinates'] = df['coordinates'].apply(lambda x: tuple(map(float, x.split(','))))

# Convert the "coordinates" column to a geometry column
df['geometry'] = df['coordinates'].apply(lambda coord: Point(coord[1], coord[0]))  # (longitude, latitude)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the CRS (Coordinate Reference System) to WGS84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

,Name of Project,Developer(s),coordinates,Status,Caption,Total square footage,Office square footage,Office space preleased,Link,geometry
0,Giller Tower,Giller family,"(25.812843946932976, -80.13520530046938)","In June, the Miami Beach Design Review Board a...",The Giller family plans a mixed-use project on...,NaN,NaN,NaN,https://therealdeal.com/miami/2024/06/05/ira-g...,POINT (-80.13521 25.81284)
1,The Fifth Miami Beach,"Sumaida + Khurana, Bizzi+Bilgili. Former Googl...","(25.774032963985963, -80.13801069619636)",Under construction. Completion is expected nex...,Sumaida + Khurana and Bizzi+Bilgili are develo...,"60,200 sf",NaN,"About 10,000 sf",https://therealdeal.com/miami/2023/12/27/bizzi...,POINT (-80.13801 25.77403)
2,NaN,"Sumaida + Khurana, with partners","(25.773892628650724, -80.13974700627581)","On April 25, the Miami Beach Planning Board vo...",Sumaida + Khurana also plans a five-story buil...,NaN,NaN,NaN,https://therealdeal.com/miami/2024/03/28/sumai...,POINT (-80.13975 25.77389)
3,One Soundscape Park,Shvo,"(25.791455592403146, -80.13144604965949)",Shvo's planned five-story One Soundscape Park ...,Shvo's planned five-story One Soundscape Park ...,"62,500 sf",NaN,NaN,https://therealdeal.com/miami/2024/02/01/micha...,POINT (-80.13145 25.79146)
4,The Alton,Shvo,"(25.791265819966448, -80.14159472805648)",It scored final approval in December.,Shvo's six-story The Alton mixed-use building ...,NaN,"170,000 sf",NaN,https://therealdeal.com/miami/2023/10/11/micha...,POINT (-80.14159 25.79127)
5,NaN,"Wayne M. Boich, Bruce Beal and Andrew Mathias","(25.79516704153476, -80.14165522529154)",NaN,"Wayne M. Boich, in partnership with Bruce Beal...",NaN,NaN,NaN,https://therealdeal.com/miami/2022/12/14/miami...,POINT (-80.14166 25.79517)
6,Eighteen Sunset,"Deco Capital Group, led by Bradley Colmer, and...","(25.792690596928665, -80.14414600113716)",Under construction,Deco Capital Group and RWN Real Estate Partner...,NaN,NaN,NaN,https://therealdeal.com/miami/2021/05/14/deco-...,POINT (-80.14415 25.79269)


In [11]:
df["short_description"] = df["Caption"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Caption'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["Link "] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2024/06...
1    <a href="https://therealdeal.com/miami/2023/12...
2    <a href="https://therealdeal.com/miami/2024/03...
3    <a href="https://therealdeal.com/miami/2024/02...
4    <a href="https://therealdeal.com/miami/2023/10...
5    <a href="https://therealdeal.com/miami/2022/12...
6    <a href="https://therealdeal.com/miami/2021/05...
Name: description_link, dtype: object

In [12]:
gdf

,Name of Project,Developer(s),coordinates,Status,Caption,Total square footage,Office square footage,Office space preleased,Link,geometry
0,Giller Tower,Giller family,"(25.812843946932976, -80.13520530046938)","In June, the Miami Beach Design Review Board a...",The Giller family plans a mixed-use project on...,NaN,NaN,NaN,https://therealdeal.com/miami/2024/06/05/ira-g...,POINT (-80.13521 25.81284)
1,The Fifth Miami Beach,"Sumaida + Khurana, Bizzi+Bilgili. Former Googl...","(25.774032963985963, -80.13801069619636)",Under construction. Completion is expected nex...,Sumaida + Khurana and Bizzi+Bilgili are develo...,"60,200 sf",NaN,"About 10,000 sf",https://therealdeal.com/miami/2023/12/27/bizzi...,POINT (-80.13801 25.77403)
2,NaN,"Sumaida + Khurana, with partners","(25.773892628650724, -80.13974700627581)","On April 25, the Miami Beach Planning Board vo...",Sumaida + Khurana also plans a five-story buil...,NaN,NaN,NaN,https://therealdeal.com/miami/2024/03/28/sumai...,POINT (-80.13975 25.77389)
3,One Soundscape Park,Shvo,"(25.791455592403146, -80.13144604965949)",Shvo's planned five-story One Soundscape Park ...,Shvo's planned five-story One Soundscape Park ...,"62,500 sf",NaN,NaN,https://therealdeal.com/miami/2024/02/01/micha...,POINT (-80.13145 25.79146)
4,The Alton,Shvo,"(25.791265819966448, -80.14159472805648)",It scored final approval in December.,Shvo's six-story The Alton mixed-use building ...,NaN,"170,000 sf",NaN,https://therealdeal.com/miami/2023/10/11/micha...,POINT (-80.14159 25.79127)
5,NaN,"Wayne M. Boich, Bruce Beal and Andrew Mathias","(25.79516704153476, -80.14165522529154)",NaN,"Wayne M. Boich, in partnership with Bruce Beal...",NaN,NaN,NaN,https://therealdeal.com/miami/2022/12/14/miami...,POINT (-80.14166 25.79517)
6,Eighteen Sunset,"Deco Capital Group, led by Bradley Colmer, and...","(25.792690596928665, -80.14414600113716)",Under construction,Deco Capital Group and RWN Real Estate Partner...,NaN,NaN,NaN,https://therealdeal.com/miami/2021/05/14/deco-...,POINT (-80.14415 25.79269)


In [13]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [12]:
gdf.to_file('miami_beach_office_projects.geojson', driver='GeoJSON')

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/miami_beach_office_projects_08_12_24
